In [86]:
import math
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import dash_bootstrap_components as dbc
from sklearn.datasets import make_blobs

import warnings
warnings.filterwarnings("ignore", message="To exit: use 'exit', 'quit', or Ctrl-D.")

In [87]:
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

df = pd.DataFrame({'y': np.random.normal(loc=0, scale=10, size=1000),
                   'x': np.random.normal(loc=10, scale=2, size=1000)})

In [88]:
COLORS = {'0': "red", '1': "blue", '2': "grey"}

# Neue Datenpunkte
X = np.array([[1, 2], [3, 4], [5, 6]])  # Beispielwerte, passen Sie sie entsprechend an
y = np.array([0, 1, 2])  # Beispielwerte, passen Sie sie entsprechend an

cluster_df = pd.DataFrame(data=X, columns=["X", "Y"])
cluster_df['cluster'] = [str(i) for i in y]


In [89]:
app.layout = html.Div([
    html.Div([html.H1("Dashboard 4")], className="header"),
    html.Div([
        dcc.Tabs(id="tabs", children=[
            dcc.Tab(label='Tab One', id="tab_1_graphs", children=[html.Div([
                dbc.Row([
                    dbc.Col([dcc.Dropdown(options=['red', 'green', 'blue'], value='red', id='color', multi=False)], width=6),
                    dbc.Col([dcc.Slider(min=math.floor(df['y'].min()), max=math.ceil(df['y'].max()), id="min_value")], width=6)
                ]),
                dbc.Row([
                    dbc.Col([dcc.Graph(id="graph_1")], width=6),
                    dbc.Col([dcc.Graph(id="graph_2")], width=6)
                ])
            ], className="tab_content")]),
            dcc.Tab(label='Tab Two', id="tab_2_graphs", children=[html.Div([
                dbc.Row([
                    dbc.Col([dcc.Graph(id="graph_3", selectedData=None, relayoutData=None)], width=8),
                    dbc.Col([dcc.Graph(id="graph_4")], width=4)
                ])
            ], className="tab_content")])
        ])
    ], className="content")
])

In [90]:
@app.callback(Output("graph_1", "figure"), Input("color", "value"))
def update_graph_1(dropdown_value_color):
    fig = px.histogram(df, x="y", color_discrete_sequence=[dropdown_value_color])
    fig.update_layout(template="plotly_white")
    return fig

In [91]:
@app.callback(Output("graph_2", "figure"), Input("min_value", "value"))
def update_graph_2(min_value):
    if min_value:
        dff = df[df['y'] > min_value]
    else:
        dff = df
    fig = px.scatter(dff, x='x', y='y')
    fig.update_layout(template="plotly_white")
    return fig

In [92]:
@app.callback(Output("graph_3", "figure"), Output("graph_4", "figure"), Input("graph_3", "selectedData"))
def update_graph_3_and_4(selected_data):
    if selected_data is None or not selected_data["points"]:
        cluster_dff = cluster_df
    else:
        selected_points = [point["pointIndex"] for point in selected_data["points"]]
        cluster_dff = cluster_df.iloc[selected_points]

    fig3 = px.scatter(cluster_dff, x="X", y="Y", color="cluster", color_discrete_map=COLORS,
                      category_orders={"cluster": ["0", "1", "2"]}, height=750)
    fig3.update_layout(template="plotly_white", coloraxis_showscale=False)
    fig3.update_traces(marker=dict(size=8))

    group_counts = cluster_dff[['cluster', 'X']].groupby('cluster').count()

    fig4 = go.Figure(data=[go.Bar(x=group_counts.index, y=group_counts['X'], marker=dict(color=[COLORS.get(str(i), 'gray') for i in group_counts.index]))])

    fig4.update_layout(height=750, template="plotly_white", title="<b>Counts per cluster</b>", xaxis_title="cluster",
                       title_font_size=25)
    return fig3, fig4

In [93]:
@app.callback(Output("graph_3", "selectedData"), Output("graph_4", "selectedData"), Input("graph_3", "clickData"))
def update_selected_data(click_data):
    return click_data, None if click_data else None

if __name__ == '__main__':
    app.run_server(debug=False, port=8012)

Dash is running on http://127.0.0.1:8012/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8012
Press CTRL+C to quit
